In [1]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'language-identification-datasst:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F94159%2F218863%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240326%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240326T075212Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D7a103813235a42df3f931106d7e51b8ef6fb53fedab90a5588eafc1b44752f7d568aa5f487428a73edb517b67ffb1ae53be6e71279e07dd62c7ebb4f6d62ecaff6039366b8e2a72e26b4aaff8ca88fe182f87063d554ddaaeb6464a6ecae6821c552650177a299f77de2b82b57bd8fbd2e88ca6f94c7f3b3a0a090373e013546eeaf710672e3701c77fa28f20a6a430367c2234ab9bc6e9d80e62c2b63014ce769ffe3f99f7b531c3a2054691ae54475a35146ddea3c1676def0e4825bbeaaa0d562d6eeeb1b3503f66f01251bef28f8b90818e84bfdb965eec35b3fa05ac2f2b30b693626317ee73dd8f93a5bd93a4337d19ad1cfdcc33b3aba9134b1119e98'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 5798108 bytes downloaded
Downloaded and uncompressed: language-identification-datasst
Data source import complete.


# Preface
First of all, **thanks a lot for this fantastic dataset!**

It is very interesting, as it contains a lot of texts. Most of the rows consist of *two languages* - a **primary** one for the major part and a **secondary** one used for some words only.

According to this observation, we already have a lot of noise in our data (i.e. all words in the secondary language).

A part from that, we are *not* dealing with long texts here, all texts are relatively short.

So, we are not working on a plain vanilla problem using the 10 best text books available for each of the investigated languages, but on a *realistic scenario* using kind of real world data.

# Pupose
This notebook, should show alternative ways, how to detect the language.

The focus lies on the identification of efficient algorithms (in terms of memory requirements) by exploiting the knowledge about languages and their structures.

# Agenda
1. Get familiar with the data
2. Language specifics *aka Uni- vs. Bi-Grams*
3. Naive Bayes
4. k Nearest Neighbours
5. Sum of squared differences
6. Kulback Leibler Divergence
7. Kolmogorov Smirnov Test
8. Analysis & Comparison of F1-Scores

# Get familiar with the data

In [2]:
import pandas as pd
raw = pd.read_csv('../input/language-identification-datasst/dataset.csv')
raw

,Text,language
0,klement gottwaldi surnukeha palsameeriti ning ...,Estonian
1,sebes joseph pereira thomas på eng the jesuit...,Swedish
2,ถนนเจริญกรุง อักษรโรมัน thanon charoen krung เ...,Thai
3,விசாகப்பட்டினம் தமிழ்ச்சங்கத்தை இந்துப் பத்திர...,Tamil
4,de spons behoort tot het geslacht haliclona en...,Dutch
...,...,...
21995,hors du terrain les années et sont des année...,French
21996,ใน พศ หลักจากที่เสด็จประพาสแหลมมลายู ชวา อินเ...,Thai
21997,con motivo de la celebración del septuagésimoq...,Spanish
21998,年月，當時還只有歲的她在美國出道，以mai-k名義推出首張英文《baby i like》，由...,Chinese


In [3]:
# Languages
languages = set(raw['language'])
print('Languages', languages)
print('========')
# Examples of multiple langs taken from heads and tails
print('Swedish & English:', raw['Text'][1])
print('Thai & English:', raw['Text'][2])
print('Chinese & English:', raw['Text'][21998])

Languages {'French', 'Dutch', 'Chinese', 'Latin', 'Urdu', 'Japanese', 'Thai', 'Russian', 'Portugese', 'Spanish', 'Persian', 'Turkish', 'Indonesian', 'English', 'Romanian', 'Arabic', 'Swedish', 'Estonian', 'Korean', 'Hindi', 'Pushto', 'Tamil'}
Swedish & English: sebes joseph pereira thomas  på eng the jesuits and the sino-russian treaty of nerchinsk  the diary of thomas pereira bibliotheca instituti historici s i --   rome libris 
Thai & English: ถนนเจริญกรุง อักษรโรมัน thanon charoen krung เริ่มตั้งแต่ถนนสนามไชยถึงแม่น้ำเจ้าพระยาที่ถนนตก กรุงเทพมหานคร เป็นถนนรุ่นแรกที่ใช้เทคนิคการสร้างแบบตะวันตก ปัจจุบันผ่านพื้นที่เขตพระนคร เขตป้อมปราบศัตรูพ่าย เขตสัมพันธวงศ์ เขตบางรัก เขตสาทร และเขตบางคอแหลม
Chinese & English: 年月，當時還只有歲的她在美國出道，以mai-k名義推出首張英文《baby i like》，由美國的獨立廠牌bip·record發行，以外國輸入盤的形式在日本發售，旋即被抢购一空。其後於月日發行以倉木麻衣名義發行的首張日文單曲《love day after tomorrow》，正式於日本出道。這張單曲初動銷量只得約萬張，可是其後每週銷量一直上升，並於年月正式突破百萬銷量，合计万张。成為年最耀眼的新人歌手。


Well, if you inspect the data yourself, you will find plenty more examples containing two languages.

Now, let's do, what we always need to do when applying ML somewhere: *Split the data into train and test.*

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

X=raw['Text']
y=raw['language']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(len(X_train))
print(len(X_test))
print(len(y_train))
print(len(y_test))

# Language specifics

To be memory efficient, we need to restrict ourselves to a minimum number of features, which is still sufficient to distinguish between the languages.

## Excurs on Cryptographie
When thinking about a solution, the first thing, which came into my mind, was the lecture on Crypto-Analysis. Breaking a [Vigenère-](https://en.wikipedia.org/wiki/Vigen%C3%A8re_cipher) or [Ceasar-Cipher](https://en.wikipedia.org/wiki/Caesar_cipher) is performed by exploiting the statistical distribution of characters using [Frequency analysis](https://en.wikipedia.org/wiki/Frequency_analysis) on Uni-Grams, Bi-Grams or even Tri-Grams.

Therefore I feel confident, that using one of these n-Grams ($n \in {1,2,3}$) will be sufficient here.

## Uni-Gram

The smallest pieces of information are single characters, i.e. **Uni-Grams**.

Using Uni-Grams allows us to identify *all* languages, which consist of *unique symbols*.
E.g. this approach is sufficient to identify Chinese, Korean, Japanese, ...

Let's inspect the unigrams:

In [ ]:
# Extract Unigrams
from sklearn.feature_extraction.text import CountVectorizer
unigramVectorizer = CountVectorizer(analyzer='char', ngram_range=(1,1))
X_unigram_train_raw = unigramVectorizer.fit_transform(X_train)
X_unigram_test_raw = unigramVectorizer.transform(X_test)


unigramFeatures = unigramVectorizer.get_feature_names()

print('Number of unigrams in training set:', len(unigramFeatures))

So, we have **6816** Uni-Grams.

But how are they distributed across the languages?

In [ ]:
# Aggregate Unigrams per language
def train_lang_dict(X_raw_counts, y_train):
    lang_dict = {}
    for i in range(len(y_train)):
        lang = y_train[i]
        v = np.array(X_raw_counts[i])
        if not lang in lang_dict:
            lang_dict[lang] = v
        else:
            lang_dict[lang] += v

    # to relative
    for lang in lang_dict:
        v = lang_dict[lang]
        lang_dict[lang] = v / np.sum(v)

    return lang_dict

language_dict_unigram = train_lang_dict(X_unigram_train_raw.toarray(), y_train.values)

# Collect relevant chars per language
def getRelevantCharsPerLanguage(features, language_dict, significance=1e-5):
    relevantCharsPerLanguage = {}
    for lang in languages:
        chars = []
        relevantCharsPerLanguage[lang] = chars
        v = language_dict[lang]
        for i in range(len(v)):
            if v[i] > significance:
                chars.append(features[i])
    return relevantCharsPerLanguage

relevantCharsPerLanguage = getRelevantCharsPerLanguage(unigramFeatures, language_dict_unigram)

# Print number of unigrams per language
for lang in languages:
    print(lang, len(relevantCharsPerLanguage[lang]))

As we can see in the above overview, following languages are using a lot of unique symbols:
- Chinese:  3,249
- Japanese: 2,054
- Korean:   1,407

I.e. we can easily identify these languages by using Uni-Grams.

But:
- All other languages are using **much fewer symbols**.
- And most of the other languages **share common symbols**.

Well, the Uni-Gram-approach might also be usefull, if languages share *common symbols*, like the [Latin symbols](https://en.wikipedia.org/wiki/Latin_alphabet) which are used in a lot of [Indo-European](https://en.wikipedia.org/wiki/Indo-European_languages) languages including e.g. [Romance](https://en.wikipedia.org/wiki/Romance_languages) (Italian, Romanian, Spanish, ...) and [Germanic](https://en.wikipedia.org/wiki/Germanic_languages) (English, German, Dutch, ...). And in a similar way also for [Kyrillic letters](https://en.wikipedia.org/wiki/Cyrillic_alphabets) used for Russian, Serbian, ...

In this cases we need to use the distribution of the characters, which will allow us to distinguish between the root languages. So we will be able to distinguish between a language based on Romance (e.g. Italian) or a Germanic (e.g. Dutch).

Unfortunately, using **Uni-Grams** will not be sufficient in all cases. We might face some issues, when we try to distinguish between languages having the same root, like for the Romance languages: Italian, Spanish, French, Protuguese, Catalan, ... Or for the Germanic languages: English, Dutch, Swedish, ...

So, let's have a look on some European languages, sharing Latin symbols

In [ ]:
# get most common chars for a few European languages
europeanLanguages = ['Portugese', 'Spanish', 'Latin', 'English', 'Dutch', 'Swedish']
relevantChars_OnePercent = getRelevantCharsPerLanguage(unigramFeatures, language_dict_unigram, 1e-2)

# collect and sort chars
europeanCharacters = []
for lang in europeanLanguages:
    europeanCharacters += relevantChars_OnePercent[lang]
europeanCharacters = list(set(europeanCharacters))
europeanCharacters.sort()

# build data
indices = [unigramFeatures.index(f) for f in europeanCharacters]
data = []
for lang in europeanLanguages:
    data.append(language_dict_unigram[lang][indices])

#build dataframe
df = pd.DataFrame(np.array(data).T, columns=europeanLanguages, index=europeanCharacters)
df.index.name = 'Characters'
df.columns.name = 'Languages'

# plot heatmap
import seaborn as sn
import matplotlib.pyplot as plt
sn.set(font_scale=0.8) # for label size
sn.set(rc={'figure.figsize':(10, 10)})
sn.heatmap(df, cmap="Greens", annot=True, annot_kws={"size": 12}, fmt='.0%')# font size
plt.show()

Looking at the data above shows, that it is very hard to distinguish between the European languages. Especially between the languages based on Romance.

Anyhow, it is also very hard to distinguish between English and Italian (labeled *Latin* in the table above), since there are only a few narrow differences in the distribution.

We will inspect the performance (in terms of the F1) score in the later chapters.

********

## Bi-Gram
The next larger piece of information are **Bi-Grams**. Unfortunately, using Bi-Grams might already blow up the required resources. So we should restrict ourselves here, to use only those,
occurring most frequently.

If we restrict ourselves to the most frequently n-Grams only, we will face another challenge. For languages containing a large amount of symbols, all combinations of symbols do not occure very often (e.g. Chinese contains >3,000 symbols). This seems to be a common pattern for languages based on syllables.


In [ ]:
# number of bigrams
from sklearn.feature_extraction.text import CountVectorizer
bigramVectorizer = CountVectorizer(analyzer='char', ngram_range=(2,2))
X_bigram_raw = bigramVectorizer.fit_transform(X_train)
bigramFeatures = bigramVectorizer.get_feature_names()
print('Number of bigrams', len(bigramFeatures))

But for the languages consisting on a few letters only, using Bi-Grams is very helpfull. Since the most frequently used Bi-Grams differ a lot.

Whereas for Japanese and Chinese we can *not* find any Bi-Grams occuring at least in 1% of the cases.

In [ ]:
# top bigrams (>1%) for Spanish, Italian (Latin), English, Dutch, Chinese, Japanese, Korean
language_dict_bigram = train_lang_dict(X_bigram_raw.toarray(), y_train.values)
relevantCharsPerLanguage = getRelevantCharsPerLanguage(bigramFeatures, language_dict_bigram, significance=1e-2)
print('Spanish', relevantCharsPerLanguage['Spanish'])
print('Italian (Latin)', relevantCharsPerLanguage['Latin'])
print('English', relevantCharsPerLanguage['English'])
print('Dutch', relevantCharsPerLanguage['Dutch'])
print('Chinese', relevantCharsPerLanguage['Chinese'])
print('Japanese', relevantCharsPerLanguage['Japanese'])

Alternatively, we could also use a **mixture** of Uni-Grams and Bi-Grams, restricted on the most frequently used ones.

## Mixture of Uni-Gram & Bi-Grams
When we restrict ourselves to a limited number of features, it is important, that we will capture details for each language. Since Chinese consists of >3,000 of different symbols, the probability of the most frequently used Chinese Uni-Grams might be below the top 1000 used Bi-Grams of the other languages.

In the following, I will try some arbitrarily chosen Mixtures. They are straight-forward ideas, based on intuition only.

For a more analytic approach, one should use a dimension reduction technique or something similar.

### Mixture Uni- & Bi-Grams (using the top 1%)
So, first try, take Uni- & Bi-Grams occurring at least in 1% of all cases.

In [ ]:
# Uni- & Bi-Gram Mixture CountVectorizer for top 1% features
from sklearn.feature_extraction.text import CountVectorizer

top1PrecentMixtureVectorizer = CountVectorizer(analyzer='char', ngram_range=(1,2), min_df=1e-2)
X_top1Percent_train_raw = top1PrecentMixtureVectorizer.fit_transform(X_train)
X_top1Percent_test_raw = top1PrecentMixtureVectorizer.transform(X_test)

language_dict_top1Percent = train_lang_dict(X_top1Percent_train_raw.toarray(), y_train.values)

top1PercentFeatures = top1PrecentMixtureVectorizer.get_feature_names()
print('Length of features', len(top1PercentFeatures))
print('')

#Unique features per language
relevantChars_Top1Percent = getRelevantCharsPerLanguage(top1PercentFeatures, language_dict_top1Percent, 1e-5)
for lang in relevantChars_Top1Percent:
    print("{}: {}".format(lang, len(relevantChars_Top1Percent[lang])))

Well, we are using a lot of features per language, which might be already a good solution.

*But chosing 3,079 features is already a lot*. And therefore the calculation is still expensive.

By the way: It is already possible to start the calculation based on 3,079 features. And the delivered results seem to be sufficient.

So, how can we do better?

### Take top 50 (Uni- & Bi-Grams) per language
Well, we can restrict ourselves to the top 50 Uni- & Bi-Grams per language.

This will lead to max `22 * 50 = 1100` features.

(*Remark:* Due to simplification, I will reuse `top1PrecentMixtureVectorizer` here, since it provides at least >500 features per language)

In [ ]:
def getRelevantGramsPerLanguage(features, language_dict, top=50):
    relevantGramsPerLanguage = {}
    for lang in languages:
        chars = []
        relevantGramsPerLanguage[lang] = chars
        v = language_dict[lang]
        sortIndex = (-v).argsort()[:top]
        for i in range(len(sortIndex)):
            chars.append(features[sortIndex[i]])
    return relevantGramsPerLanguage

top50PerLanguage_dict = getRelevantGramsPerLanguage(top1PercentFeatures, language_dict_top1Percent)

# top50
allTop50 = []
for lang in top50PerLanguage_dict:
    allTop50 += set(top50PerLanguage_dict[lang])

top50 = list(set(allTop50))

print('All items:', len(allTop50))
print('Unique items:', len(top50))

**So, when dealing with the top-50-approach on these 22 languages, we will effectively use 564 features only.**

Now, let's build the data set for the models, based on our 564 features.

In [ ]:
# getRelevantColumnIndices
def getRelevantColumnIndices(allFeatures, selectedFeatures):
    relevantColumns = []
    for feature in selectedFeatures:
        relevantColumns = np.append(relevantColumns, np.where(allFeatures==feature))
    return relevantColumns.astype(int)

relevantColumnIndices = getRelevantColumnIndices(np.array(top1PercentFeatures), top50)


X_top50_train_raw = np.array(X_top1Percent_train_raw.toarray()[:,relevantColumnIndices])
X_top50_test_raw = X_top1Percent_test_raw.toarray()[:,relevantColumnIndices]

print('train shape', X_top50_train_raw.shape)
print('test shape', X_top50_test_raw.shape)

Before we start to use the features in our models, let's finally discuss the feature selection.

## n-Gram Discussion

Let's recap the above:
- using Uni-Grams is sufficient for syllable languages (Chinese, Japanese, ...)
- but Uni-Grams do not help when inspecting European languages - here we need at least some Bi-Grams
- using all Bi-Grams will blow up the memory required
- the same is true for **Tri-Grams** - so, we will do *no* further investigation on that here

Conclusion: **From a theoretical perspective, it is most efficient to use a Mixture of the most common Uni-Grams and Bi-Grams**.

***

# Naive Bayes
Now, we can apply the Naive Bayes on our different feature sets:
- Unigram (`X_unigram_train_raw`)
- Mixture Top 1% (`X_top1Percent_train_raw`)
- Mixture Top 50 (`X_top50_train_raw`)

First of all, we need to define some utility functions for our evaluation.

In [ ]:
# Define some functions for our purpose

from sklearn.preprocessing import normalize
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix, f1_score
import seaborn as sn
import matplotlib.pyplot as plt
import scipy

# Utils for conversion of different sources into numpy array
def toNumpyArray(data):
    data_type = type(data)
    if data_type == np.ndarray:
        return data
    elif data_type == list:
        return np.array(data_type)
    elif data_type == scipy.sparse.csr.csr_matrix:
        return data.toarray()
    print(data_type)
    return None


def normalizeData(train, test):
    train_result = normalize(train, norm='l2', axis=1, copy=True, return_norm=False)
    test_result = normalize(test, norm='l2', axis=1, copy=True, return_norm=False)
    return train_result, test_result

def applyNaiveBayes(X_train, y_train, X_test):
    trainArray = toNumpyArray(X_train)
    testArray = toNumpyArray(X_test)

    clf = MultinomialNB()
    clf.fit(trainArray, y_train)
    y_predict = clf.predict(testArray)
    return y_predict

def plot_F_Scores(y_test, y_predict):
    f1_micro = f1_score(y_test, y_predict, average='micro')
    f1_macro = f1_score(y_test, y_predict, average='macro')
    f1_weighted = f1_score(y_test, y_predict, average='weighted')
    print("F1: {} (micro), {} (macro), {} (weighted)".format(f1_micro, f1_macro, f1_weighted))

def plot_Confusion_Matrix(y_test, y_predict, color="Blues"):
    allLabels = list(set(list(y_test) + list(y_predict)))
    allLabels.sort()
    confusionMatrix = confusion_matrix(y_test, y_predict, labels=allLabels)
    unqiueLabel = np.unique(allLabels)
    df_cm = pd.DataFrame(confusionMatrix, columns=unqiueLabel, index=unqiueLabel)
    df_cm.index.name = 'Actual'
    df_cm.columns.name = 'Predicted'

    sn.set(font_scale=0.8) # for label size
    sn.set(rc={'figure.figsize':(15, 15)})
    sn.heatmap(df_cm, cmap=color, annot=True, annot_kws={"size": 12}, fmt='g')# font size
    plt.show()



## Uni-Grams

Remember, we are working on **6,816** features in the case of Uni-Grams.

In [ ]:
# Unigrams
X_unigram_train, X_unigram_test = normalizeData(X_unigram_train_raw, X_unigram_test_raw)
y_predict_nb_unigram = applyNaiveBayes(X_unigram_train, y_train, X_unigram_test)
plot_F_Scores(y_test, y_predict_nb_unigram)
plot_Confusion_Matrix(y_test, y_predict_nb_unigram, "Oranges")

A score of 92% for F1 (weighted) is not too bad. Anyhow it is not sufficient and we will do better.

And as suggested, it is pretty hard to distinguish between languages of the same familiy.

Here our major problem is to distinguish between Dutch, English and Swedish.

## Top 1% Mixture

In the case of top 1% 1- & 2-Grams we are working on 3,079 features.

In [ ]:
# Top 1%
X_top1Percent_train, X_top1Percent_test = normalizeData(X_top1Percent_train_raw, X_top1Percent_test_raw)
y_predict_nb_top1Percent = applyNaiveBayes(X_top1Percent_train, y_train, X_top1Percent_test)
plot_F_Scores(y_test, y_predict_nb_top1Percent)
plot_Confusion_Matrix(y_test, y_predict_nb_top1Percent, "Reds")

F1 (weighted) of 97.46% is much better. Again we have some issues with European languages here.

Unfortunately, we are still working on a lot of features.

Let's see, if the Mixture of the Top 50 will perform better.

## Top 50 Mixture
We are using **564** features only.

In [ ]:
# Top 50
X_top50_train, X_top50_test = normalizeData(X_top50_train_raw, X_top50_test_raw)
y_predict_nb_top50 = applyNaiveBayes(X_top50_train, y_train, X_top50_test)
plot_F_Scores(y_test, y_predict_nb_top50)
plot_Confusion_Matrix(y_test, y_predict_nb_top50, "Greens")

Well, the result for F1 (weighted) of 97.52% is somehow better, but not significantly.

Anyway we are using much fewer features. Therefore I will prefer this approach.

## Comparison
For Naive Bayes we achieve the following scores for F1 (weighted):
- Unigram: 0.9201996483569281 (using 6,816 features)
- Top 1% Mixture: 0.9746025107937131 (3,079 features)
- Top 50 Mixture: 0.9751510056992273 (564 features)

Now let's see, how k-Nearest-Neighbours performs.

# k Nearest Neighbour
First, we need to define the apply method and to discuss the size of k.

The default for k=5. Let's stick to that default.

I will run kNN on the Top-50-feature-set only. (If you like to run it on the Uni-Grams please uncomment below.)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

def applyNearestNeighbour(X_train, y_train, X_test):
    trainArray = toNumpyArray(X_train)
    testArray = toNumpyArray(X_test)

    clf = KNeighborsClassifier()
    clf.fit(trainArray, y_train)
    y_predict = clf.predict(testArray)
    return y_predict


## Unigrams
#y_predict_knn_unigram = applyNearestNeighbour(X_unigram_train, y_train, X_unigram_test)
#plot_F_Scores(y_test, y_predict_knn_unigram)
#plot_Confusion_Matrix(y_test, y_predict_knn_unigram, "Purples")

# Top 50
y_predict_knn_top50 = applyNearestNeighbour(X_top50_train, y_train, X_top50_test)
plot_F_Scores(y_test, y_predict_knn_top50)
plot_Confusion_Matrix(y_test, y_predict_knn_top50, "Blues")

The results of kNN are almost equal to those of the Naive Bayes approach.

In a future notebook, I might investigate some Auto-ML or hyperparameter optimization using the classical scikit algorithms.

But, right now I am more interested in some analytical algorithms.


# Analytical Algorithms

*Is it possible to use an classical analytical algorithm to predict the language?*

Well, this should work, since we are working on something very similar to a distribution.

If we would restrict ourselves to a fixed n (for the n-Grams) only, it would be a real distribution.

But for the Mixture case, this does not hold. E.g. the occurence of 'e' and 'n' and 'en' will not be independent.

Anyhow, I will convert all inputs into a relative form, so that we do not run into technical problems.

Lets inspect, if the following algorithms work here:
- Least-Squares
- Kulback-Leibler Divergence
- Kolmogorov-Smirnov-Test



## Assumptions
- We are working on discrete distributions
- $p_l(i)$ is the probability function of language $l$ for character/symbol $i$
- $q(i)$ is the probability function of the current sample


In [ ]:
# Train lang dict
def toRelative(X_test):
    return [v / np.sum(v) for v in X_test]

language_dict_top50 = train_lang_dict(X_top50_train, y_train.values)
X_top50_test_rel = toRelative(X_top50_test)

`language_dict_top50` contains the relative distribution per language for the Top-50-feature-set. ($p_l(i)$ for all languages $l$)

`X_top50_test_rel` contains (distribution-like) relative values for the test data, based on the Top-50-feature-set. ($q(i)$ for all samples)

# (Ordinary) least squares
Identify the language by taking the minimum of the sum of the squared differnces per feature (=character/symbol), i.e.

$$ \delta_l = \sum _i (p_l(i)-q(i))^2 $$
$$ l^* = argmin_l(\delta_l) $$

In [ ]:
def ols_predict(language_dict, X_test):
    def calcSquareDifference(p, q):
        return np.sum((p-q)**2)

    def ols(language_dict, v, langs):
        olsVector = np.array([calcSquareDifference([language_dict[l]], v) for l in langs])
        index = np.argmin(olsVector)
        return langs[index]

    langs = [l for l in language_dict]
    return [ols(language_dict, v, langs) for v in X_test]


ols_predictions = ols_predict(language_dict_top50, X_top50_test_rel)

plot_F_Scores(y_test, ols_predictions)
plot_Confusion_Matrix(y_test, ols_predictions, 'cool')

It seems, that OLS results in more differences to the labelled data than Naive Bayes and k-Nearest-Neighbour.

Differences are:
- Latin -> predicted as English (seems to be a common error also for Naive Bayes and kNN)
- Urdu -> predicted as Arabic
- Chinese -> English
- Spanish -> Portuguese


# Kullback Leibler Divergence
Taking the smallest value of [Kullback Leibler Divergence](https://en.wikipedia.org/wiki/Kullback%E2%80%93Leibler_divergence) as a measure for the difference between distributions, given by:
$$D_\text{KL}(p \parallel q) = \sum_{x\in\mathcal{X}} p(x) \log\left(\frac{p(x)}{q(x)}\right)$$

In [ ]:
# Kullback Leibler Divergence
from math import log


def kl_predict(language_dict, X_test):

    def kl_divergence(p, q):
        p_ = np.array(p) + 1e-200
        q_ = np.array(q) + 1e-200
        n = len(p)
        return np.sum([p_[i] * log(p_[i]/ (q_[i] )) for i in range(n)])

    def predict(language_dict, v, langs):
        divs = [kl_divergence(language_dict[l], v) for l in langs]
        index = np.argmin(divs)
        return langs[index]

    langs = [l for l in language_dict]
    return [predict(language_dict, v, langs) for v in X_test]


kl_predictions = kl_predict(language_dict_top50, X_top50_test_rel)

plot_F_Scores(y_test, kl_predictions)
plot_Confusion_Matrix(y_test, kl_predictions, 'plasma')

In addition to the observed errors for OLS, the KL approach has some more errors on:
- Pushto vs. Persian
- Persian vs. Urdu
- Latin vs. French

# Kolmogorov Smirnov Test

Use the two-sample-case of the [Kolmogorov Smirnov Test](https://en.wikipedia.org/wiki/Kolmogorov%E2%80%93Smirnov_test#Discrete_and_mixed_null_distribution) to check, if a given sample is written in a specific language.

The general formula is given by:

$$ D_{n,m}=\sup_x |F_{1,n}(x)-F_{2,m}(x)| $$

We then reject the Null hypothesis ($H_0$) at level $\alpha$ if
$$ D_{n,m}>c(\alpha)\sqrt{\frac{n + m}{n\cdot m}}. $$

Where
- $F_1$ and $F_2$ are two samples
- consisting of $m$ respective $n$ samples

* Applied in our case:

$$ D^l_{n,m}=\sup_i |P(i)-Q(i)| $$

for each language $l$.


We have in total 1,000 texts per language (20% = 800 for training). So, $ n \approx 800 * AvgChars $

$\alpha=0.01 \implies c(\alpha)=1.628 $

Since we might reject the Null hypothesis for all languages, we could also cover the case, that the sample is written in a completely different language.

In the case when $H_0$ holds for more languages, we will use the language $l$ with the smallest value for $D^l$.

In other words: *We will choose the language, where the maximum of all absolute differences is the smallest. Except the Null hypothesis does not hold.*

In [ ]:
# Calculate average number of chars per Text
avgChars = np.mean(raw.apply(lambda x : len(x['Text']), axis=1))
print('avgChars', avgChars)

In [ ]:
def ks_predict(language_dict, X_test, n, c_alpha=1.628):
    def calcMaxAbsDifference(p, q):
        return np.max(np.abs((p-q)))

    def scaleAlpha(n, m, c_alpha):
        factor = ((n + m) / n / m) ** 0.5
        return factor * c_alpha

    def ks(language_dict, v, langs):
        ksVector = np.array([calcMaxAbsDifference([language_dict[l]], v) for l in langs])
        index = np.argmin(ksVector)
        m = len(v)
        scaledAlpha = scaleAlpha(n, m, c_alpha)
        if ksVector[index] <= scaledAlpha:
            return langs[index]
        else:
            return '_N/A_'

    langs = [l for l in language_dict]
    return [ks(language_dict, v, langs) for v in X_test]


ks_predictions = ks_predict(language_dict_top50, X_top50_test_rel, 356*800)
print('None-Predicitons:', (np.array(ks_predictions)=='_N/A_').sum())
plot_F_Scores(y_test, ks_predictions)
plot_Confusion_Matrix(y_test, ks_predictions, 'summer')

In addition to the problems observed for OLS and KL we are facing a lot of more problems here.


# Comparison of results

Before we finally compare all results, we start with the implementation of a benchmark using an existing library. And have a quick look on the mismatched data.


## Benchmark library `langdetect`

Unfortunately, we need to add some fix-up code:
- the [output](https://pypi.org/project/langdetect/) of `langdetect` is in iso-639 format (except for Chinese).
- Italian is called Latin in our dataset
- Portuguese is named Portugese in our dataset

In [ ]:
! pip install langdetect
! pip install iso-639

from langdetect import detect
from iso639 import languages

def proofLanguage(text):
    twoLetterCode = detect(text)[:2] #consolidate Chinese
    if twoLetterCode == 'it': # Italian -> Latin
        return 'Latin'
    elif twoLetterCode == 'pt': # Portuguese -> Portugese
        return 'Portugese'
    else:
        lang = languages.get(alpha2=twoLetterCode)
        langName = lang.name
        if not langName==None:
            return langName
        return twoLetterCode

y_test_proof = [proofLanguage(t) for t in X_test]

In [ ]:
plot_F_Scores(y_test, y_test_proof)
plot_Confusion_Matrix(y_test, y_test_proof, 'cool')

Obviously the benchmark library has some large differences in the prediction:
- Pushto and Persian are both predicted to be Persian.
- Some Italian (here Latin) is classified as Catalan, English, French, Romanian, ...
- Chinese is also predicted as Korean.
- Texts labelled to be in Urdu are classified to be Arabic.


***

Well, the benchmark seems to be beaten by far. But this does not reflect the reality.

Let us have a look on the occured differences.



## Error Analysis
Lets have a brief look on the error details for some selected scenarios.

First we define a helper method `plotTopErrors`:

In [ ]:
def plotTopErrors(y_predict, top=5):
    ys = y_test.values
    Xs = X_test.values
    errorCount = 0

    for i in range(len(ys)):
        if not ys[i]==y_predict[i]:
            errorCount += 1
            print("#{}: Expected: {}, Predicted: {}".format(errorCount, ys[i], y_predict[i]))
            print("Text:", Xs[i])
            print("=================================================")
        if errorCount >= top:
            break

Now, lets start with the error inspection.

### Naive Bayes on Mix Top 50

In [ ]:
plotTopErrors(y_predict_nb_top50, top=5)

Well, there are errors on both sides. But as far as I can see, the majority of the predicted values is correct.

So far I would suggest, that the real F1 is better than 97%.

Anyhow, I need to proof my suggestion in the future.

### Naive Bayes on Uni-Grams

In [ ]:
plotTopErrors(y_predict_nb_unigram, top=5)

Using Uni-Grams, does result in the initially expected errors.

Well, it is very hard to distinguish between similar languages.

### Kolmogorov Smirnov on Mix Top 50

In [ ]:
plotTopErrors(ks_predictions, top=5)

Well, it seems that a lot of the predicted values are wrong for KS.

## Comparison Ranking by F1 weighted
1. 0.9751510056992273 **NB** on **Mix Top 50**
1. 0.9746025107937131 **NB** on **Mix Top 1%**
1. 0.9723506120636130 **kNN** on **Mix Top 50**
1. 0.9645429434449447 **KL** on **Mix Top 50**
1. 0.9495364489332023 **OLS** on **Mix Top 50**
1. 0.9201996483569281 **NB** on **Unigrams**
1. 0.8789520584453827 **langdetect**
1. 0.8025836015759978 **KS** on **Mix Top 50**

# Future Work & Next Steps
- Rerun analysis on cleaned data
- Use quantitative method to identify features, e.g. PCA
- Improve results by Hyperparameter-Optimzation (at least run some Auto-ML)
- Try to identify, if there are two languages. And try to predict the secondary language